In [ ]:
%load_ext autoreload
%autoreload 2

# Loading Data

In [2]:
import loader
data_loader = loader.load_data()

# Create the classifier and train it

In [12]:
import classifier
import lightning as L
model = classifier.NeuralNetwork()
trainer = L.Trainer(accelerator="auto", devices="auto", strategy="auto")
trainer.fit(model, data_loader['train'], data_loader['val'])

KeyboardInterrupt: 

# Test the classifier

In [ ]:
trainer.test(model, data_loader['test'])